In [1]:
import os
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
import json
env = json.load(open("env.json"))
os.environ = env

2024-06-22 17:22:27.672502: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 17:22:27.699577: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 17:22:28.286656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ebozik/anaconda3/envs/nlp2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
data = pd.read_json("data/reviews.json.gz")
data.dropna(inplace=True)
data

,review_text,sentiment
2,Doposiaľ najrýchlejšie dodanie tovaru,1.0
3,Neskora donaśka,0.0
4,Dopravca s nemožnosťou platby kartou,0.0
6,Blizko,1.0
7,Rýchle dodanie tovaru. Som veľmi spokojná.,1.0
...,...,...
3048269,"cena, ochota, informácie, dostupnosť",1.0
3048270,prijal by som viacej titulov na dvd a cd,0.0
3048272,"kvalitná ponuka, promptný osobny odber",1.0
3048274,má všetko čo potrebujem,1.0


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-06-22 17:22:35.035159: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-22 17:22:35.059509: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-22 17:22:35.059542: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
data.loc[:, 'review_text'] = data['review_text'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
data = data.loc[data["review_text"] != '']
data.loc[:, "sentiment"] = data["sentiment"].astype('int8')
data.reset_index(inplace=True, drop=True)

In [5]:
data

,review_text,sentiment
0,doposiaľ najrýchlejšie dodanie tovaru,1.0
1,neskora donaśka,0.0
2,dopravca s nemožnosťou platby kartou,0.0
3,blizko,1.0
4,rýchle dodanie tovaru som veľmi spokojná,1.0
...,...,...
1793207,cena ochota informácie dostupnosť,1.0
1793208,prijal by som viacej titulov na dvd a cd,0.0
1793209,kvalitná ponuka promptný osobny odber,1.0
1793210,má všetko čo potrebujem,1.0


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['review_text'].values)
X = tokenizer.texts_to_sequences(data['review_text'].values)
X = pad_sequences(X)

In [8]:
Y = data['sentiment'].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [18]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.models import Sequential
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
batch_size = 256
epochs = 5

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
# with tf.device('/CPU:0'):   
#     model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])

Epoch 1/5
5604/5604 ━━━━━━━━━━━━━━━━━━━━ 3881s 692ms/step - accuracy: 0.9499 - loss: 0.1349 - val_accuracy: 0.9600 - val_loss: 0.1103
Epoch 2/5
5604/5604 ━━━━━━━━━━━━━━━━━━━━ 3871s 691ms/step - accuracy: 0.9614 - loss: 0.1072 - val_accuracy: 0.9624 - val_loss: 0.1051
Epoch 3/5
5604/5604 ━━━━━━━━━━━━━━━━━━━━ 3872s 691ms/step - accuracy: 0.9636 - loss: 0.1017 - val_accuracy: 0.9628 - val_loss: 0.1035
Epoch 4/5
5604/5604 ━━━━━━━━━━━━━━━━━━━━ 3873s 691ms/step - accuracy: 0.9648 - loss: 0.0980 - val_accuracy: 0.9635 - val_loss: 0.1018
Epoch 5/5
5604/5604 ━━━━━━━━━━━━━━━━━━━━ 3876s 692ms/step - accuracy: 0.9661 - loss: 0.0953 - val_accuracy: 0.9637 - val_loss: 0.1011
Restoring model weights from the end of the best epoch: 5.


In [23]:
model.save("lstm_heaureka.keras")

In [24]:
import pickle
with open("tokenizers/lstm_heureka_tokenizer.pickle", "wb") as file:
    pickle.dump(tokenizer, file)

In [25]:
X.shape[1]

450

In [27]:
def predict_sentiment(text):
    text = text.lower().replace(r'[^\w\s]', '')
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=450)
    pred = model.predict(padded)
    return 'Positive' if np.argmax(pred) == 1 else 'Negative'

In [29]:
import numpy as np
texts = [
    "Akokoľvek sa na to pozriem tak to nevyzerá dobre.",
    "Tento deň je skvelý",
    "Neviem či som správne pochopil.",
    "To si vážne urobila?!",
    "To myslíš vážne?",
    "Ja som veľmi rád, že si sa to konečne naučila.",
    "Som pekný.",
    "Zajtra si kúpim pivo.",
    "Tak teraz neviem čo ďalej.",
    "Prehrali sme ale vidím na tom aj pozitívum.",
    "Tak som rozmýšlal, že to asi aj výjde."
]
for i in texts:
    print(i)
    print(predict_sentiment(i))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
Positive


In [30]:
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Test score:", score)
print("Test accuracy:", acc)

1401/1401 - 319s - 227ms/step - accuracy: 0.9637 - loss: 0.1011
Test score: 0.10110335052013397
Test accuracy: 0.9636574387550354


In [31]:
data

,review_text,sentiment
0,doposiaľ najrýchlejšie dodanie tovaru,1.0
1,neskora donaśka,0.0
2,dopravca s nemožnosťou platby kartou,0.0
3,blizko,1.0
4,rýchle dodanie tovaru som veľmi spokojná,1.0
...,...,...
1793207,cena ochota informácie dostupnosť,1.0
1793208,prijal by som viacej titulov na dvd a cd,0.0
1793209,kvalitná ponuka promptný osobny odber,1.0
1793210,má všetko čo potrebujem,1.0


In [36]:
    print(predict_sentiment("viac slovenskych pobočiek by bodlo"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
Negative
